In [7]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import pandas as pd
import numpy

In [8]:
# Define the number of input features
input_dim = 13

# Define the encoding dimension (number of features in the encoded representation)
encoding_dim = 4

# Input layer
input_layer = Input(shape=(input_dim,))

In [11]:
# Encoding layers
encoded = Dense(8, activation='relu')(input_layer)
encoded = Dense(4, activation='relu')(input_layer)
encoded = Dense(2, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# Decoding layers
decoded = Dense(2, activation='sigmoid')(encoded)
decoded = Dense(4, activation='sigmoid')(decoded)
decoded = Dense(8, activation='sigmoid')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

In [18]:
# Autoencoder model
autoencoder = Model(input_layer, decoded)

# Encoder model (for extracting encoded representations)
encoder = Model(input_layer, encoded)

# Decoder model (for reconstructing inputs from encoded representations)
encoded_input = Input(shape=(encoding_dim,))
decoder_layer1 = autoencoder.layers[-4]
decoder_layer2 = autoencoder.layers[-3]
decoder_layer3 = autoencoder.layers[-2]
decoder_layer4 = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer4(decoder_layer3(decoder_layer2(decoder_layer1(encoded_input)))))

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_absolute_percentage_error')

In [19]:
emissions = pd.read_csv('emissions.csv').drop('Entity', axis=1).drop('Year', axis=1)
emissions = emissions.to_numpy()
(emissions_test, emissions_train) = numpy.split(emissions, [int(.2*len(emissions))])

In [20]:
# Train the autoencoder
autoencoder.fit(emissions_train, emissions_train,
                epochs=50,
                batch_size=32,
                shuffle=True,
                validation_data=(emissions_test, emissions_test))

# Encode and decode some data
encoded_data = encoder.predict(emissions_test)
decoded_data = decoder.predict(encoded_data)

# Print the original, encoded, and decoded data
print("Original data: ", emissions_test[:5])
print("Encoded data: ", encoded_data[:5])
print("Decoded data: ", decoded_data[:5])

Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 52335780.0000 - val_loss: 29705614.0000
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 53556368.0000 - val_loss: 29579816.0000
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 47134612.0000 - val_loss: 29292884.0000
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 48899684.0000 - val_loss: 28543976.0000
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 44262288.0000 - val_loss: 26694164.0000
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43325400.0000 - val_loss: 22974750.0000
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37329460.0000 - val_loss: 18064380.0000
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30493736.0000 - val_loss: 13673445.0000
Epoch 9/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24206804.0000 - val_loss: 10494777.0000
Epoch 10/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17289256.0000 - val_loss: 8259329.5000
Epoch 11/50
48/48 ━━